# Содержание:

  
    3) Метод Hashing с классификаторами:
            1) Ridge Classifier
            2) Perceptron
            3) Passive Agressive
            4) BeurnolliNB
            5) MultinomialNB
            6) KNeighbors
            7) RandomForest
            8) NearestCentroid
            9) SGDClassifier
            10) LinearSVC
    4) Визуализация


In [2]:
import os
from lxml import etree
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

import numpy as np
import scipy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
from pymystem3 import Mystem

## Загружаю данные:

In [4]:
from Levenshtein import distance

In [5]:
f = open('./out.csv', 'r')
file_hash_all = {}
for line in f:
    attrs = line.split(';')
    path = attrs[0]
    class_name = attrs[8]
    if ' ' in class_name:
        class_name = class_name.split()[0]
    if '|' in class_name:
        class_name = class_name.split('|')[0]
    file_name = os.path.basename(path)
    dir_name = os.path.dirname(path)
    full_dir = os.path.join('/Users/Valeriya/Desktop/CoursePaper/corpus', dir_name)
    files = os.listdir(full_dir)
    if file_name in files:
        file_hash_all[file_name] = class_name
    elif file_name + '.xml' in files:
        file_hash_all[file_name + '.xml'] = class_name
    elif file_name + '.xhtml' in files:
        file_hash_all[file_name + '.xhtml'] = class_name
    else:
        edit = 1000
        true_name = ''
        for file_in_dir in files:
            dist = distance(file_name, file_in_dir)
            if dist < edit:
                edit = dist
                true_name = file_in_dir
        file_hash_all[true_name] = class_name

In [6]:
from collections import defaultdict

In [7]:
f = open('./out.csv', 'r')
# cl = {'отчет':0,'поздравление':0, 'интервью':0, 'репортаж':0, 'хроника':0, 'объявление':0, 'комментарий':0,
#       'совет':0, 'анонс':0, 'заметка':0, 'информационное сообщение':0, 'статья':0}
cl = {'статья':0, 'комментарий':0, 'интервью':0, 'объявление':0, 'поздравление':0}
# cl = {'отчет':0,'поздравление':0}
file_hash = defaultdict(int)
file_all = defaultdict(int)
for file_name, class_name in file_hash_all.items():
    if class_name in cl:
        cl[class_name] += 1
#         if cl[class_name] <= 150:
        file_hash[file_name] = class_name
    file_all[class_name] +=1
#     elif class_name == 'репортаж' or class_name == 'отчет':
#         cl['other'] += 1
#         file_hash[file_name] = 'other'

In [7]:
q = 0

for i in file_all:
    q+=1
print(q)

71


In [8]:
t = 0
for i in file_all:
    t+= file_all[i]

In [9]:
print(file_all)

defaultdict(<class 'int'>, {'': 2575, 'указ': 7, 'репортаж': 331, 'доклад': 2, 'объявление': 384, 'заявление': 2, 'план': 1, 'совет': 121, 'биография': 9, 'комментарии': 173, 'аннотация': 15, 'инструкция': 17, 'рассказ': 3, 'резолюция': 1, 'календарь': 3, 'рецепт': 21, 'хроника': 284, 'интервью': 834, 'надпись': 8, 'буклет': 1, 'подбор': 1, 'памфлет': 2, 'поздравление': 198, 'гороскоп': 4, 'игра': 8, 'письмо': 94, 'type': 1, 'дневник,': 10, 'стихотворение': 21, 'подпись': 1, 'пресс-конференция': 1, 'обзор': 50, 'речь': 2, 'беседа': 8, 'закон': 5, 'публицистика': 7, 'лекция': 1, 'статья': 2503, 'консультация': 3, 'предисловие,': 2, 'монография': 1, 'отчет': 576, 'очерк': 108, 'анкета': 1, 'решение': 1, 'некролог': 17, 'заметка': 3715, 'положение': 8, 'рецензия': 90, 'пресс-релиз': 4, 'отзыв': 22, 'фельетон': 4, 'проповедь': 1, 'обращение': 29, 'комментарий': 674, 'предисловие': 2, 'постановление': 89, 'комикс': 6, 'тест': 3, 'правила': 3, 'информационное': 1034, 'анонс': 227, 'эссе': 14

In [8]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            if filename in file_hash:
                class_text = file_hash[filename]
                tree = etree.parse(open_name)
                yield tree, class_text, open_name

In [9]:
m = Mystem()

def mystem_tokenizer(string):
    tokens = m.lemmatize(string)
    word_tokens = []
    for token in tokens:
        # if token.isalnum():
        if token.isalpha():
            word_tokens.append(token)
    return word_tokens
    
# CountVectorizer(tokenizer=mystem_tokenizer)

In [10]:
texts = []
for tree, class_text, open_name in read_txt('/Users/Valeriya/Desktop/CoursePaper/corpus'):
    body = tree.xpath('//body')
    text = '\n'.join([elem.text for elem in body[0] if elem.text])
#     if class_text != 'поздравление':
#         class_text = 'другие'
    if text:
        texts.append((text, class_text, open_name))
    else:
        print(open_name)

/Users/Valeriya/Desktop/CoursePaper/corpus/rossija/nesnyat/south/sochi/451404.xhtml


In [11]:
len(texts)

4592

In [12]:
features = {}
# with open('formal_features_3.tsv', 'r') as f:
with open('formal_features_3.tsv', 'r') as f:
    for line in f:
        items = line.rstrip().split('\t')
        path = items[0]
        feats = items[1::]
        features[path] = feats

In [13]:
classes = [text[1] for text in texts]

In [14]:
len(classes)

4592

In [15]:
texts_2 = [text[0] for text in texts]

In [16]:
data = []
for i, (text, class_name, open_name) in enumerate(texts):
    open_name = open_name.replace('/Users/Valeriya/Desktop/CoursePaper/', './')
    formal_features = features[open_name][::]
    n_excl = 0
    for item in text[0]:
        if item == '!':
            n_excl+=1

    n_ques = 0
    for item in text[0]:
        if item == '?':
            n_ques+=1
    formal_features.append(n_excl)
    formal_features.append(n_ques)
    
    formal_features = [float(x) for x in formal_features]
    formal_features = formal_features[:2:] + [x/formal_features[0] for x in formal_features[2::]]
    
    data.append(formal_features)

In [22]:
# vectorizer = HashingVectorizer(non_negative=True, n_features=32)
# X_train_hash = vectorizer.transform(X_train)
# X_test_hash = vectorizer.transform(X_test)

In [17]:
vectorizer = HashingVectorizer(non_negative=True, n_features=32)
data_hash = vectorizer.fit_transform(texts_2)

In [18]:
final = []
for i, form_vec in enumerate(data):
    sparse_form = scipy.sparse.csr_matrix(form_vec)
    sparse_both = scipy.sparse.hstack([sparse_form, data[i]])
    final.append(sparse_both)

data_sparse = scipy.sparse.vstack(final)

In [27]:
# vectorizer_tf = TfidfVectorizer(sublinear_tf=True, max_df=0.5, ngram_range=(1,3))
# vectorizer = HashingVectorizer(non_negative=True, n_features=32)

# data_hash = vectorizer.fit_transform(data_sparse)
# X_train_hash = vectorizer.transform(X_train)
# X_test_hash = vectorizer.transform(X_test)

In [52]:
# for i, j in zip(data, data_hash):
#     final.append(i+list(j.toarray().flatten()))

In [53]:
# X_train, X_test, y_train, y_test = train_test_split(final, classes, test_size=0.3)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(data_sparse, classes, test_size=0.3)

In [30]:
# vectorizer = HashingVectorizer(non_negative=True, n_features=32)
# X_train_hash = vectorizer.fit_transform(X_train)
# X_test_hash = vectorizer.transform(X_test)

## Метод tf-idf

In [20]:
# vectorizer_tf = TfidfVectorizer(sublinear_tf=True, max_df=0.5)
# X_train_tf = vectorizer_tf.fit_transform(X_train)
# X_test_tf = vectorizer_tf.transform(X_test)
# print("Extracting best features by a chi-squared test")

ch2 = SelectKBest(chi2, k=30)
X_train_ch = ch2.fit_transform(X_train, y_train)
X_test_ch = ch2.transform(X_test)

In [33]:
from xgboost import XGBClassifier
import xgboost as xgb

ImportError: No module named 'xgboost'

## Классификаторы


In [98]:
# def benchmark(clf):
#     print('_' * 80)
#     print("Training: ")
#     print(clf)
# #     t0 = time()
#     if clf == MultinomialNB(alpha=.01):
#         clf.fit(X_train_ch, y_train)
# #         clf.fit(X_train, y_train)
#     else:
#         clf.fit(X_train_ch, y_train)
# #         clf.fit(X_train_norm, y_train)


# #     t0 = time
#     if clf == MultinomialNB(alpha=.01):
#         pred = clf.predict(X_test_ch)
# #         pred = clf.predict(X_test)
#     else:
#         pred = clf.predict(X_test_ch)
# #         pred = clf.predict(X_test_norm)

#     score = metrics.accuracy_score(y_test, pred)
#     print("accuracy:   %0.3f" % score)
#     print()
    
#     report = classification_report(y_test, pred)
#     print()
#     print(report)
    
#     matrix = confusion_matrix(y_test, pred)
#     print()
#     print(matrix)
    
#     clf_descr = str(clf).split('(')[0]
#     if "penalty" in vars(clf):
#         return clf_descr+' '+str(clf.penalty), score
#     else:
#         return clf_descr, score

# results = []

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import AdaBoostClassifier

# clf1 = RandomForestClassifier(n_estimators = 350, bootstrap=False)
# results.append(benchmark(clf1))

# # make some plots

# indices = np.arange(len(results))

# results = [[x[i] for x in results] for i in range(2)]

________________________________________________________________________________
Training: 
RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=350, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
accuracy:   0.753


             precision    recall  f1-score   support

   интервью       0.74      0.32      0.45       254
комментарий       0.86      0.52      0.65       204
     статья       0.74      0.96      0.84       745

avg / total       0.76      0.75      0.72      1203


[[ 81  12 161]
 [  8 107  89]
 [ 21   6 718]]


In [21]:
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    clf.fit(X_train, y_train)

    pred = clf.predict(X_test)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)

    print()
    clf_descr = str(clf).split('(')[0]
    report = classification_report(y_test, pred)
    print()
    print(report)
    return clf_descr, score


results = []
for clf, name in (
        (RidgeClassifier(tol=1e-2, solver="lsqr"), "Ridge Classifier"),
        (Perceptron(n_iter=50), "Perceptron"),
        (PassiveAggressiveClassifier(n_iter=50), "Passive-Aggressive"),
        (KNeighborsClassifier(n_neighbors=10), "kNN"),
        (RandomForestClassifier(n_estimators=100), "Random forest")):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf))

for penalty in ["l2", "l1"]:
    print('=' * 80)
    print("%s penalty" % penalty.upper())
    # Train Liblinear model
    results.append(benchmark(LinearSVC(loss='l2', penalty=penalty,
                                            dual=False, tol=1e-3)))

    # Train SGD model
    results.append(benchmark(SGDClassifier(alpha=.0001, n_iter=50,
                                           penalty=penalty)))

# Train SGD with Elastic Net penalty
print('=' * 80)
print("Elastic-Net penalty")
results.append(benchmark(SGDClassifier(alpha=.0001, n_iter=50,
                                       penalty="elasticnet")))

# Train NearestCentroid without threshold
print('=' * 80)
print("NearestCentroid (aka Rocchio classifier)")
results.append(benchmark(NearestCentroid()))

# Train Logistic Regression
print('=' * 80)
print("Logistic Regression")
results.append(benchmark(LogisticRegression()))

# Train sparse Naive Bayes classifiers
print('=' * 80)
print("Naive Bayes")
results.append(benchmark(MultinomialNB(alpha=.01)))
results.append(benchmark(BernoulliNB(alpha=.01)))

print('=' * 80)
print("LinearSVC with L1-based feature selection")
# The smaller C, the stronger the regularization.
# The more regularization, the more sparsity.
results.append(benchmark(Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC( penalty="l1", dual=False, tol=1e-3))),
  ('classification', LinearSVC())
])))

# make some plots

indices = np.arange(len(results))

results = [[x[i] for x in results] for i in range(4)]

# clf_names, score = results

Ridge Classifier
________________________________________________________________________________
Training: 
RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='lsqr',
        tol=0.01)


/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/linear_model/ridge.py:311: UserWarning: In Ridge, only 'sag' solver can currently fit the intercept when X is sparse. Solver has been automatically changed into 'sag'.
  warnings.warn("In Ridge, only 'sag' solver can currently fit the "


accuracy:   0.533


              precision    recall  f1-score   support

    интервью       0.00      0.00      0.00       263
 комментарий       0.00      0.00      0.00       212
  объявление       0.00      0.00      0.00       120
поздравление       0.00      0.00      0.00        49
      статья       0.53      1.00      0.70       734

 avg / total       0.28      0.53      0.37      1378

Perceptron
________________________________________________________________________________
Training: 
Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=50, n_jobs=1, penalty=None, random_state=0, shuffle=True,
      verbose=0, warm_start=False)


/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy:   0.234


              precision    recall  f1-score   support

    интервью       0.28      0.52      0.36       263
 комментарий       0.21      0.88      0.34       212
  объявление       0.00      0.00      0.00       120
поздравление       0.00      0.00      0.00        49
      статья       0.00      0.00      0.00       734

 avg / total       0.09      0.23      0.12      1378

Passive-Aggressive
________________________________________________________________________________
Training: 
PassiveAggressiveClassifier(C=1.0, class_weight=None, fit_intercept=True,
              loss='hinge', n_iter=50, n_jobs=1, random_state=None,
              shuffle=True, verbose=0, warm_start=False)


...

/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy:   0.536


              precision    recall  f1-score   support

    интервью       0.00      0.00      0.00       263
 комментарий       0.00      0.00      0.00       212
  объявление       0.00      0.00      0.00       120
поздравление       0.14      0.88      0.25        49
      статья       0.65      0.95      0.77       734

 avg / total       0.35      0.54      0.42      1378

NearestCentroid (aka Rocchio classifier)
________________________________________________________________________________
Training: 
NearestCentroid(metric='euclidean', shrink_threshold=None)
accuracy:   0.310


              precision    recall  f1-score   support

    интервью       0.30      0.46      0.36       263
 комментарий       0.18      0.38      0.25       212
  объявление       0.16      0.16      0.16       120
поздравление       0.22      0.82      0.35        49
      статья       0.69      0.23      0.34       734

 avg / total       0.47      0.31      0.32      1378

Logist

/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy:   0.623


              precision    recall  f1-score   support

    интервью       0.56      0.02      0.04       263
 комментарий       0.57      0.06      0.10       212
  объявление       0.81      0.68      0.74       120
поздравление       0.89      0.67      0.77        49
      статья       0.60      0.99      0.75       734

 avg / total       0.62      0.62      0.51      1378

Naive Bayes
________________________________________________________________________________
Training: 
MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
accuracy:   0.442


              precision    recall  f1-score   support

    интервью       0.35      0.37      0.35       263
 комментарий       0.22      0.29      0.25       212
  объявление       0.30      0.28      0.29       120
поздравление       0.22      0.82      0.35        49
      статья       0.71      0.51      0.60       734

 avg / total       0.51      0.44      0.46      1378

__________________________________

IndexError: tuple index out of range